# Task

Extract random twitter data for the last one week, a maximum of 200,000 rows. This data should not have any specific hash tags.

# Methodology

Using Snscrape which is a scraper for social networking services (SNS). It scrapes details like user profiles, hashtags, or searches and returns the discovered items, e.g. the relevant posts.

Installing and importing the necessary libraries.

Setting Variables to be used

We will be scraping a minimum number of 200,000 random tweets from twitter. The date is set from 1st Decwmber 2022 to 7th December 2022 in the span of one week. since my Ram isnot enough to scrape it all at once it was then done in batches (from 1st-2nd, 3rd-4th, 5th-6th, 4th-7th, 5th-7th, 6th-7th)

Finally merging the different dataframe to a single one and as a CSV file for further analysis

In [3]:
!pip install snscrape

In [4]:
!pip install wordcloud
!pip install emot

In [11]:
#libraries needed
import pandas as pd
import snscrape.modules.twitter as sntwitter
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
# nltk.download('stopwords') #run once and comment it out to avoid it downloading multiple times
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

import string
import re
import textblob
from textblob import TextBlob

from wordcloud import WordCloud, STOPWORDS
from emot.emo_unicode import UNICODE_EMOJI

porter = PorterStemmer()

lemmatizer = WordNetLemmatizer() 

from wordcloud import ImageColorGenerator
from PIL import Image

import warnings
%matplotlib inline

# Tweets scrapping

The tweets were scrapped in batches to get the total of over 200000 tweets

In [40]:
# creating a query for the search

query = "geocode:9.0820,8.6753,923768km since:2022-12-05 until:2022-12-07 lang:en"
tweets = []
for i, tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
    if i>50000:
        break
    else:
        tweets.append([tweet.date, tweet.id, tweet.url, tweet.user.username, tweet.sourceLabel, tweet.user.location, tweet.content, tweet.likeCount, tweet.retweetCount])
df = pd.DataFrame(tweets, columns = ['Date', 'ID', 'url', 'username', 'source', 'location', 'tweet', 'num_of_likes', 'num_of_retweet'])
df.to_csv('raw_tweet1241.csv', mode = 'a')

# Importing the File into pandas DataFrames:

Seeing that the tweets was scrapped into 5 different files, we need to merge all into a single dataframe for analyis

In [41]:
#we paste all the files in a single directory
#import the modules
import os
#read the path
file_path = "C:\\Users\Visacheck Admin\\Desktop\\SGA_DSCI_2\\Random_tweets_files"#list all the files from the directory
file_list = os.listdir(file_path)
file_list

['raw_tweet123.csv',
 'raw_tweet1234.csv',
 'raw_tweet1235.csv',
 'raw_tweet1236.csv',
 'raw_tweet1237.csv',
 'raw_tweet1239.csv',
 'raw_tweet1240.csv',
 'raw_tweet1241.csv']

Now to read multiple CSV files with the similar table structure, you can use pandas.DataFrame.append() OR pd.concat() functions.

Let’s look at the 3 sample CSV files we’ll be working with.

In [42]:
#read the csv file into a dataframe
df1 = pd.read_csv("C:\\Users\Visacheck Admin\\Desktop\\SGA_DSCI_2\\Random_tweets_files\\raw_tweet123.csv")
df2 = pd.read_csv("C:\\Users\Visacheck Admin\\Desktop\\SGA_DSCI_2\\Random_tweets_files\\raw_tweet1234.csv")
df3 = pd.read_csv("C:\\Users\Visacheck Admin\\Desktop\\SGA_DSCI_2\\Random_tweets_files\\raw_tweet1235.csv")
df4 = pd.read_csv("C:\\Users\Visacheck Admin\\Desktop\\SGA_DSCI_2\\Random_tweets_files\\raw_tweet1236.csv")
df5 = pd.read_csv("C:\\Users\Visacheck Admin\\Desktop\\SGA_DSCI_2\\Random_tweets_files\\raw_tweet1237.csv")
df6 = pd.read_csv("C:\\Users\Visacheck Admin\\Desktop\\SGA_DSCI_2\\Random_tweets_files\\raw_tweet1239.csv")
df7 = pd.read_csv("C:\\Users\Visacheck Admin\\Desktop\\SGA_DSCI_2\\Random_tweets_files\\raw_tweet1240.csv")
df8 = pd.read_csv("C:\\Users\Visacheck Admin\\Desktop\\SGA_DSCI_2\\Random_tweets_files\\raw_tweet1241.csv")

print('All files read successfully')

All files read successfully


In [43]:
#check the shapes of the files

df1.shape
print(df1.shape)

df2.shape
print(df2.shape)

df3.shape
print(df3.shape)

df4.shape
print(df4.shape)

df5.shape
print(df5.shape)

df6.shape
print(df6.shape)

df7.shape
print(df7.shape)

df8.shape
print(df8.shape)

(20001, 10)
(20001, 10)
(22434, 10)
(22432, 10)
(24024, 10)
(20001, 10)
(50001, 10)
(46676, 10)


This gives a total of 205,562

In [44]:
df.columns

Index(['Date', 'ID', 'url', 'username', 'source', 'location', 'tweet',
       'num_of_likes', 'num_of_retweet'],
      dtype='object')

Another way to do this is by using glob to list all the csv in the directory

first, we have to go into the directory which has the files

In [71]:
cd

C:\Users\Visacheck Admin


In [72]:
cd Desktop

C:\Users\Visacheck Admin\Desktop


In [73]:
cd SGA_DSCI_2

C:\Users\Visacheck Admin\Desktop\SGA_DSCI_2


In [74]:
cd Random_tweets_files

C:\Users\Visacheck Admin\Desktop\SGA_DSCI_2\Random_tweets_files


In [75]:
import glob
import pandas as pd#list all csv files only
csv_files = glob.glob('*.{}'.format('csv'))
csv_files

['raw_tweet123.csv',
 'raw_tweet1234.csv',
 'raw_tweet1235.csv',
 'raw_tweet1236.csv',
 'raw_tweet1237.csv',
 'raw_tweet1239.csv',
 'raw_tweet1240.csv',
 'raw_tweet1241.csv']

# Combining multiple files with the similar table structure using pandas.DataFrame.append()

In [76]:
df_append = pd.DataFrame()#append all files together
for file in csv_files:
            df_temp = pd.read_csv(file)
            df_append = df_append.append(df_temp, ignore_index=True)
df_append

C:\Users\Visacheck Admin\AppData\Local\Temp\ipykernel_19716\3281929373.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_append = df_append.append(df_temp, ignore_index=True)
C:\Users\Visacheck Admin\AppData\Local\Temp\ipykernel_19716\3281929373.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_append = df_append.append(df_temp, ignore_index=True)
C:\Users\Visacheck Admin\AppData\Local\Temp\ipykernel_19716\3281929373.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_append = df_append.append(df_temp, ignore_index=True)
C:\Users\Visacheck Admin\AppData\Local\Temp\ipykernel_19716\3281929373.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas

,Unnamed: 0,Date,ID,url,username,source,location,tweet,num_of_likes,num_of_retweet
0,0,2022-12-01 23:59:52+00:00,1598466944507723777,https://twitter.com/Legacybozz1/status/1598466...,Legacybozz1,Twitter for Android,"Abuja, Nigeria",@BoySpyce 😁🙏🙏🙏,0,0
1,1,2022-12-01 23:59:33+00:00,1598466865394745349,https://twitter.com/IsmailAliyuAbd3/status/159...,IsmailAliyuAbd3,Twitter for Android,Nigeria,"But alas, kudiratu was the lucky chirp yeah? h...",0,0
2,2,2022-12-01 23:59:26+00:00,1598466837796249600,https://twitter.com/LekkyFrosh1/status/1598466...,LekkyFrosh1,Twitter for Android,"Lagos, Nigeria",@Objlynks @BNXN @BNXN i really want to meet yo...,0,0
3,3,2022-12-01 23:59:25+00:00,1598466833438412802,https://twitter.com/LukenIsrael/status/1598466...,LukenIsrael,Twitter for Android,NaN,@kelemoshane Long life and prosperity,0,0
4,4,2022-12-01 23:59:25+00:00,1598466831827828736,https://twitter.com/growthmindsetll/status/159...,growthmindsetll,Twitter for Android,"Brooklyn, New York",ingredient in creative achievement. And it was...,2,1
...,...,...,...,...,...,...,...,...,...,...
225565,46671,2022-12-05 00:00:25+00:00,1599554248366178305,https://twitter.com/GoldenH13843226/status/159...,GoldenH13843226,Twitter for Android,NaN,@01gX439m2SvGo0 Yez He's Coming ObiDatti,1,0
225566,46672,2022-12-05 00:00:19+00:00,1599554224177545217,https://twitter.com/marvix_music/status/159955...,marvix_music,Twitter for Android,NaN,Prayer for a better day https://t.co/UWYvaZf8WM,0,0
225567,46673,2022-12-05 00:00:19+00:00,1599554220176289792,https://twitter.com/tunji_adetomiwa/status/159...,tunji_adetomiwa,Twitter for Android,"Lagos, Nigeria",Nowhere in the world do politicians make a lot...,0,0
225568,46674,2022-12-05 00:00:11+00:00,1599554187477458946,https://twitter.com/___Muktar/status/159955418...,___Muktar,Twitter for iPhone,Abuja || kaduna,@hajjokamal Ameen thank you,0,0


In [83]:
df_append.shape

(225570, 10)

In [85]:
df_append.columns

Index(['Unnamed: 0', 'Date', 'ID', 'url', 'username', 'source', 'location',
       'tweet', 'num_of_likes', 'num_of_retweet'],
      dtype='object')

In [88]:
#dropping unnamed column
to_drop = ['Unnamed: 0']
 
df_append.drop(to_drop, inplace=True, axis=1)

df_append.head()

,Date,ID,url,username,source,location,tweet,num_of_likes,num_of_retweet
0,2022-12-01 23:59:52+00:00,1598466944507723777,https://twitter.com/Legacybozz1/status/1598466...,Legacybozz1,Twitter for Android,"Abuja, Nigeria",@BoySpyce 😁🙏🙏🙏,0,0
1,2022-12-01 23:59:33+00:00,1598466865394745349,https://twitter.com/IsmailAliyuAbd3/status/159...,IsmailAliyuAbd3,Twitter for Android,Nigeria,"But alas, kudiratu was the lucky chirp yeah? h...",0,0
2,2022-12-01 23:59:26+00:00,1598466837796249600,https://twitter.com/LekkyFrosh1/status/1598466...,LekkyFrosh1,Twitter for Android,"Lagos, Nigeria",@Objlynks @BNXN @BNXN i really want to meet yo...,0,0
3,2022-12-01 23:59:25+00:00,1598466833438412802,https://twitter.com/LukenIsrael/status/1598466...,LukenIsrael,Twitter for Android,NaN,@kelemoshane Long life and prosperity,0,0
4,2022-12-01 23:59:25+00:00,1598466831827828736,https://twitter.com/growthmindsetll/status/159...,growthmindsetll,Twitter for Android,"Brooklyn, New York",ingredient in creative achievement. And it was...,2,1


In [90]:
df_append.to_csv('random_cleaned_tweets.csv', mode = 'a')